In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re
import string
import nltk
import spacy
import gensim
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from nltk.probability import FreqDist
from sklearn.svm import LinearSVC
from textblob import TextBlob
from gensim.models import Word2Vec

In [ ]:
reviews_df = pd.read_csv('amazon_vfl_reviews.csv', encoding="UTF-8")

In [ ]:
reviews_df.head()

,asin,name,date,rating,review
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product...My hair falling increase too ...
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...


In [ ]:
reviews_df.shape

(2782, 5)

In [ ]:
reviews_df.isnull().sum()

,0
asin,0
name,0
date,0
rating,0
review,6


In [ ]:
reviews_df = reviews_df.dropna()
reviews_df = reviews_df.reset_index(drop=True)

In [ ]:
def cleanText(input_string):
    modified_string = re.sub('[^A-Za-z0-9]+', ' ', input_string)
    return modified_string
reviews_df['review'] = reviews_df.review.apply(cleanText)

In [ ]:
reviews_df['review'][0]

'I bought this hair oil after viewing so many good comments But this product is not good enough First of all it s Expensive Second thing the amount of the product is low half bottle YES The bottle is not completely filled with oil If you cheating on your customers Mamaearth trust me on this you can t fool people more than once Now I know that your Brand is not good enough I am not going to buy any product from your Brand again Thumbs down for mamaearth onion oil '

In [ ]:
reviews_df['review'][110]

'Frankly I have ordered this product after seeing so many good reviews and here is my honest review after using for about 15 days Does not break my skin feel light and mositurized does nothing for my marks or uneven toneSo the purpose of fading marks or making glowing skin is not at all entertained so far Yet I want to wait till 1 bottle use Can be used as light mositurising serum for upcoming summers Hopes it helps '

In [ ]:
reviews_df['brandName'] = reviews_df['name'].str.split('-').str[0]
reviews_df.head()

,asin,name,date,rating,review,brandName
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth


In [ ]:
reviews_df['brandName'].value_counts()

,count
brandName,
Mamaearth,200
Godrej,200
Titan,200
Himalaya,200
Cinthol,200
Streax,188
Society,182
Tata,180
Dettol,180


In [ ]:
reviews_df['brandName'] = reviews_df['brandName'].str.title()
reviews_df.brandName.unique()

array(['Mamaearth', 'Godrej', 'Titan', 'Maaza', 'Paper', 'Indiana',
       'Coca', 'Natural', 'Maggi', 'Glucon', 'Amul', 'Patanjali',
       'Dettol', 'Savlon', 'Cinthol', 'Britannia', 'Nutrichoice',
       'Streax', 'Himalaya', 'Society', 'Tata', 'Fastrack', 'Reflex',
       'Mysore'], dtype=object)

In [ ]:
reviews_df.head()

,asin,name,date,rating,review,brandName
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth


In [ ]:
products = []
for value in reviews_df['name']:
    index = len(value.split('-')[0])+1
    products.append(value[index:])
reviews_df['product'] = products

In [ ]:
reviews_df.head()

,asin,name,date,rating,review,brandName,product
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth,Onion-Growth-Control-Redensyl
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth,Onion-Growth-Control-Redensyl
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth,Onion-Growth-Control-Redensyl
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth,Onion-Growth-Control-Redensyl
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth,Onion-Growth-Control-Redensyl


In [ ]:
reviews_df['clean_review_text'] = reviews_df['review'].str.lower()

In [ ]:
import string

In [ ]:
reviews_df['clean_review_text']=reviews_df['clean_review_text'].str.translate(str.maketrans('','',string.punctuation))

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopWords = stopwords.words('english') + ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']

In [ ]:
def removeStopWords(stopWords, review_text):
    newtext = ' '.join([word for word in review_text.split() if word not in stopWords])
    return newtext

In [ ]:
reviews_df['clean_review_text'] = [removeStopWords(stopWords,x) for x in reviews_df['clean_review_text']]

In [ ]:
reviews_df.head()

,asin,name,date,rating,review,brandName,product,clean_review_text
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth,Onion-Growth-Control-Redensyl,bought hair oil viewing many good comments pro...
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth,Onion-Growth-Control-Redensyl,used mama earth newly launched onion oil twice...
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth,Onion-Growth-Control-Redensyl,bad product hair falling increase much order s...
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth,Onion-Growth-Control-Redensyl,product smells similar navarathna hair oil str...
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth,Onion-Growth-Control-Redensyl,trying different onion oil hair hair healthy p...


In [ ]:
tokenList = []

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
reviews_df.to_csv('preprocessed_data.csv', index=False)

In [ ]:
for index in range(len(reviews_df)):
       tokenList.append(word_tokenize(reviews_df['clean_review_text'][index]))
reviews_df['review_tokens'] = tokenList

In [ ]:
reviews_df.head()

,asin,name,date,rating,review,brandName,product,clean_review_text,review_tokens
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth,Onion-Growth-Control-Redensyl,bought hair oil viewing many good comments pro...,"[bought, hair, oil, viewing, many, good, comme..."
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth,Onion-Growth-Control-Redensyl,used mama earth newly launched onion oil twice...,"[used, mama, earth, newly, launched, onion, oi..."
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth,Onion-Growth-Control-Redensyl,bad product hair falling increase much order s...,"[bad, product, hair, falling, increase, much, ..."
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth,Onion-Growth-Control-Redensyl,product smells similar navarathna hair oil str...,"[product, smells, similar, navarathna, hair, o..."
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth,Onion-Growth-Control-Redensyl,trying different onion oil hair hair healthy p...,"[trying, different, onion, oil, hair, hair, he..."


In [ ]:
# VADER (Valence Aware Dictionary and sentiment Reasoner)
# VADER is a sentiment analysis tool that uses a lexicon of words with associated sentiment scores.

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
sentiment_model = SentimentIntensityAnalyzer()
sentiment_scores = []
sentiment_score_flag = []

In [ ]:
#The compound value reflects the overall sentiment ranging from -1 being very negative and +1 being very positive.

In [ ]:
for text in reviews_df['clean_review_text']:
        sentimentResults = sentiment_model.polarity_scores(text)
        sentiment_score = sentimentResults["compound"]

        sentiment_scores.append(sentiment_score)

        if sentimentResults['compound'] >= 0.05 :
            sentiment_score_flag.append('positive')

        elif sentimentResults['compound'] <= - 0.05 :
            sentiment_score_flag.append('negative')

        else :
            sentiment_score_flag.append('neutral')

In [ ]:
reviews_df['scores']=sentiment_scores
reviews_df['sentiment'] = sentiment_score_flag

In [ ]:
reviews_df.head()

,asin,name,date,rating,review,brandName,product,clean_review_text,review_tokens,scores,scoreStatus,sentiment
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth,Onion-Growth-Control-Redensyl,bought hair oil viewing many good comments pro...,"[bought, hair, oil, viewing, many, good, comme...",0.7037,positive,positive
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth,Onion-Growth-Control-Redensyl,used mama earth newly launched onion oil twice...,"[used, mama, earth, newly, launched, onion, oi...",0.8689,positive,positive
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth,Onion-Growth-Control-Redensyl,bad product hair falling increase much order s...,"[bad, product, hair, falling, increase, much, ...",-0.6820,negative,negative
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth,Onion-Growth-Control-Redensyl,product smells similar navarathna hair oil str...,"[product, smells, similar, navarathna, hair, o...",-0.2732,negative,negative
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth,Onion-Growth-Control-Redensyl,trying different onion oil hair hair healthy p...,"[trying, different, onion, oil, hair, hair, he...",0.9286,positive,positive


In [ ]:
reviews_df['sentiment'].value_counts()

,count
sentiment,
positive,2168
negative,390
neutral,218


In [ ]:
reviews_df.info()

NameError: name 'reviews_df' is not defined